In [1]:
state = 'WV'    # 2-letter state code
k = 2           # number of districts
level = 'tract' # 'county' or 'tract'

In [2]:
# Read graph from the json file "<state>_<level>.json"
filepath = 'C:\\districting-data-2020\\'
filename = state + '_' + level + '.json'

# GerryChain has a built-in function for reading graphs of this type:
from gerrychain import Graph
G = Graph.from_json( filepath + filename )

for node in G.nodes:
    G.nodes[node]['TOTPOP'] = G.nodes[node]['P0010001']

In [3]:
# Let's impose a 1% population deviation (+/-0.5%)
deviation = 0.01
total_population = sum( G.nodes[node]['TOTPOP'] for node in G.nodes )

import math
L = math.ceil( ( 1 - deviation / 2 ) * total_population / k )
U = math.floor( ( 1 + deviation / 2 ) * total_population / k )
print("Using L =",L,"and U =",U,"and k =",k)

Using L = 892374 and U = 901342 and k = 2


In [4]:
import gurobipy as gp
from gurobipy import GRB

# create model 
m = gp.Model()

import networkx as nx
DG = nx.DiGraph(G)   # directed version of G

# create variables
# x[i,j] equals one when county i is assigned to district j
x = m.addVars(G.nodes, k, vtype=GRB.BINARY)  

# y[u,v,j] equals one when arc (u,v) is cut because u->j but not v->j
y = m.addVars(DG.edges, k, vtype=GRB.BINARY) 

# z[j] is inverse Polsby-Popper score for district j
z = m.addVars(k)

# A[j] = area of district j
A = m.addVars(k)

# P[j] = perimeter of district j
P = m.addVars(k)

for j in range(k):
    z[j].lb = 0
    A[j].lb = 0
    P[j].lb = 0

Set parameter Username
Academic license - for non-commercial use only - expires 2023-01-01


In [5]:
# objective is to minimize sum of inverse Polsby-Popper scores
coef = 1 / ( 2 * math.pi )
m.setObjective( coef * gp.quicksum( z[j] for j in range(k) ), GRB.MINIMIZE )

In [6]:
# add SOCP constraints relating inverse Polsby-Popper score z[j] to area and perimeter
m.addConstrs( P[j] * P[j] <= 2 * A[j] * z[j] for j in range(k) )

# add constraints on areas A[j] 
m.addConstrs( A[j] == gp.quicksum( G.nodes[i]['area'] * x[i,j] for i in G.nodes ) for j in range(k) )

# add constraints on perimeters P[j]
for j in range(k):
    m.addConstr( P[j] == gp.quicksum( DG.edges[u,v]['shared_perim'] * y[u,v,j] for u,v in DG.edges )
             + gp.quicksum( G.nodes[i]['boundary_perim'] * x[i,j] for i in G.nodes if G.nodes[i]['boundary_node'] ) )
    
m.update()

In [7]:
# add constraints saying that each county i is assigned to one district
m.addConstrs( gp.quicksum( x[i,j] for j in range(k)) == 1 for i in G.nodes )

# add constraints saying that each district has population at least L and at most U
m.addConstrs( gp.quicksum( G.nodes[i]['TOTPOP'] * x[i,j] for i in G.nodes) >= L for j in range(k) )
m.addConstrs( gp.quicksum( G.nodes[i]['TOTPOP'] * x[i,j] for i in G.nodes) <= U for j in range(k) )

# add constraints saying that edge {u,v} is cut if u is assigned to district j but v is not.
m.addConstrs( x[u,j] - x[v,j] <= y[u,v,j] for u,v in DG.edges for j in range(k) )

m.update()

In [8]:
# Now, let's add contiguity constraints and re-solve the model.
# We will use the contiguity constraints of Hojny et al. (MPC, 2021)
#   https://link.springer.com/article/10.1007/s12532-020-00186-3

# Add root variables: r[i,j] equals 1 if node i is the "root" of district j
r = m.addVars( G.nodes, k, vtype=GRB.BINARY)

# To solve the MIP faster, fix a district root:
r[0,0].LB = 1

# Add flow variables: f[u,v] = amount of flow sent across arc uv 
#  Flows are sent across arcs of the directed version of G which we call DG

f = m.addVars( DG.edges )

In [9]:
# The big-M proposed by Hojny et al.
M = G.number_of_nodes() - k + 1

# Each district j should have one root
m.addConstrs( gp.quicksum( r[i,j] for i in G.nodes ) == 1 for j in range(k) )

# If node i is not assigned to district j, then it cannot be its root
m.addConstrs( r[i,j] <= x[i,j] for i in G.nodes for j in range(k) ) 

# if not a root, consume some flow.
# if a root, only send out (so much) flow.
m.addConstrs( gp.quicksum( f[j,i] - f[i,j] for j in G.neighbors(i) ) 
             >= 1 - M * gp.quicksum( r[i,j] for j in range(k) ) for i in G.nodes )

# do not send flow across cut edges
m.addConstrs( f[u,v] + f[v,u] <= M * ( 1 - gp.quicksum( y[u,v,j] for j in range(k) ) ) for u,v in DG.edges )

m.update()

In [ ]:
# solve IP model
m.optimize()

Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 10774 rows, 10770 columns and 49078 nonzeros
Model fingerprint: 0xf6dc5af1
Model has 2 quadratic constraints
Variable types: 2866 continuous, 7904 integer (7904 binary)
Coefficient statistics:
  Matrix range     [3e-05, 8e+03]
  QMatrix range    [1e+00, 2e+00]
  Objective range  [2e-01, 2e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 9e+05]
Presolve removed 1100 rows and 1094 columns
Presolve time: 0.29s
Presolved: 9674 rows, 9676 columns, 44145 nonzeros
Presolved model has 2 quadratic constraint(s)
Variable types: 2866 continuous, 6810 integer (6810 binary)

Root relaxation: objective 0.000000e+00, 6535 iterations, 1.36 seconds (0.39 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.00

 14061 10360    9.40846   30  291          -    6.54533      -   133  598s
 14296 11011   10.43977  132   48          -    6.54533      -   134  607s
 15004 11402   10.74557  252   11          -    6.54533      -   132  618s
 15471 11629   12.52373  306   82          -    6.60129      -   132  626s
 15855 11771    7.23836   19  635          -    6.60129      -   132  635s
 15999 12000    8.25096   21  595          -    6.60129      -   133  645s
 16268 12304    8.39430   22  637          -    6.60129      -   134  653s
 16650 12492    9.80922   28  577          -    6.60129      -   133  665s
 16874 12659    9.05496   29  288          -    6.60129      -   132  674s
 17068 12735    9.55089   94   58          -    6.60129      -   134  681s
 17166 12864    9.84062   96   81          -    6.60129      -   136  690s
 17295 13130   10.00406   96   86          -    6.60129      -   137  699s
 17613 13185    9.96002   96   87          -    6.60129      -   138  707s
 17681 13368    9.45597  

 54415 43755   12.46489   96  204          -    7.24930      -   138 1778s
 55148 44052    7.88315   22  518          -    7.24930      -   137 1788s
 55551 44327    9.69615   26  474          -    7.24930      -   137 1798s
 55911 44457   11.97446   34  350          -    7.24930      -   137 1806s
 56043 44742   12.46404   75   75          -    7.26759      -   137 1813s
 56363 45198    9.83983   33  562          -    7.26759      -   137 1821s
 56859 45744   12.74389   59  146          -    7.26759      -   137 1831s
 57516 46398   16.38086  104  174          -    7.28033      -   136 1842s
 58450 46609    8.59643   29  534          -    7.28033      -   135 1850s
 58744 46773   10.42654   32  576          -    7.28033      -   135 1856s
 58908 46942   10.92722   49   61          -    7.28033      -   136 1866s
 59097 47321   11.02346   59   34          -    7.28033      -   136 1876s
 59521 47608   11.14011   99  122          -    7.28033      -   136 1885s
 59840 48038   11.18151  

 94502 76089   12.94728   68  162          -    7.52334      -   138 2855s
 94856 76498   14.43795  126  155          -    7.52334      -   138 2866s
 95354 76908   22.15243  194  188          -    7.52334      -   138 2877s
 95886 77155   27.47476  262  127          -    7.53344      -   138 2885s
 96177 77360   10.61321   33  120          -    7.53344      -   138 2895s
 96385 77515   10.88734   51   39          -    7.53344      -   139 2906s
 96570 77730   10.89919   72   26          -    7.53344      -   139 2916s
 96849 77796   11.50713   91  477          -    7.53344      -   139 2935s
 96947 77943   11.49439   91  123          -    7.53344      -   139 2946s
 97112 78196   12.57045  103  126          -    7.53584      -   140 2957s
 97403 78390    8.32595   25  524          -    7.53584      -   140 2965s
 97599 78660   12.02843   36  184          -    7.53584      -   140 2978s
 97879 79206   12.18196   87   59          -    7.53584      -   140 2990s
 98474 79499   12.52476  

 133463 109027   12.15959   64  137          -    7.68648      -   144 3977s
 133701 109370    9.57713   25  550          -    7.68648      -   144 3988s
 134121 109641   11.21465   48  101          -    7.68648      -   144 3997s
 134452 109741   11.33415   86   17          -    7.68648      -   144 4008s
 134575 109910   11.42649   94   61          -    7.68648      -   145 4019s
 134752 110057   11.44040  124  332          -    7.68648      -   145 4030s
 134922 110322   11.62803  133   72          -    7.68648      -   145 4038s
 135246 110676   12.96269  171   76          -    7.69592      -   145 4048s
 135657 110900   12.01720   56  514          -    7.69592      -   145 4058s
 135921 111012   12.07919   81   27          -    7.69592      -   145 4069s
 135972 111012 infeasible  227               -    7.69592      -   145 4070s
 136056 111174   12.30452   96  210          -    7.69592      -   145 4078s
 136218 111409   12.79484  113   39          -    7.69592      -   146 4088s

...

 3352913 948205    9.22566  138  118    9.35906    9.12594  2.49%   124 154335s
 3353458 948567    9.18800   61   50    9.35906    9.12595  2.49%   124 154397s
 3354236 948767    9.21511   76   70    9.35906    9.12596  2.49%   124 154455s
 3354880 949011    9.28807  142   46    9.35906    9.12597  2.49%   124 154508s
 3355527 949304     cutoff  103         9.35906    9.12598  2.49%   124 154564s
 3356169 949540     cutoff   42         9.35906    9.12599  2.49%   124 154611s
 3356737 949845    9.34326   75   58    9.35906    9.12599  2.49%   124 154677s
 3357408 950064    9.23214   62   74    9.35906    9.12600  2.49%   124 154731s
 3358045 950213    9.34583  118  109    9.35906    9.12601  2.49%   124 154780s
 3358662 950407    9.31987   41   95    9.35906    9.12601  2.49%   124 154833s
 3359207 950632     cutoff   64         9.35906    9.12602  2.49%   124 154887s
 3359852 950770    9.15299   73   55    9.35906    9.12603  2.49%   124 154931s
 3360330 951005    9.17032   59  199    

 3410668 971044     cutoff  106         9.35906    9.12657  2.48%   124 159545s
 3410957 971258    9.22121   67   36    9.35906    9.12658  2.48%   124 159601s
 3411638 971613    9.30887  126    2    9.35906    9.12659  2.48%   124 159669s
 3412335 971859 infeasible  106         9.35906    9.12659  2.48%   124 159725s
 3412951 972163    9.15274  173    9    9.35906    9.12660  2.48%   124 159797s
 3413671 972356     cutoff   98         9.35906    9.12661  2.48%   124 159851s
 3414091 972557    9.15055   98  115    9.35906    9.12661  2.48%   124 159906s
 3414607 972808    9.33878   83   59    9.35906    9.12662  2.48%   124 159952s
 3415057 973003    9.21926  106   33    9.35906    9.12662  2.48%   124 160009s
 3415685 973157    9.19617   85   83    9.35906    9.12663  2.48%   124 160057s
 3416135 973534    9.21549  155   53    9.35906    9.12663  2.48%   124 160126s
 3416943 973732    9.24454   91   30    9.35906    9.12664  2.48%   124 160180s
 3417599 973956     cutoff   90         

 3468815 994838    9.26006   68   25    9.35906    9.12721  2.48%   123 165168s
 3469415 994980    9.16895   66   88    9.35906    9.12721  2.48%   123 165220s
 3469869 995189    9.21084   71   29    9.35906    9.12722  2.48%   123 165272s
 3470438 995364 infeasible  139         9.35906    9.12722  2.48%   123 165341s
 3471053 995674    9.21208   61   31    9.35906    9.12723  2.48%   123 165404s
 3471254 995674    9.20138   74   72    9.35906    9.12723  2.48%   123 165405s
 3471803 995941    9.18867  100   54    9.35906    9.12724  2.48%   123 165467s
 3472357 996281    9.26211   77   55    9.35906    9.12725  2.48%   123 165535s
 3473057 996596    9.22901  163   52    9.35906    9.12725  2.48%   123 165600s
 3473701 996731    9.14609   93   80    9.35906    9.12727  2.48%   123 165653s
 3474281 997042     cutoff   73         9.35906    9.12728  2.48%   123 165715s
 3475027 997223    9.22775   77   66    9.35906    9.12729  2.48%   123 165768s
 3475448 997472    9.19589   92  225    

 3524964 1017984    9.13922  123   86    9.35906    9.12780  2.47%   122 170751s
 3525521 1018278    9.31269  131   19    9.35906    9.12781  2.47%   122 170811s
 3526190 1018494    9.29763   79   18    9.35906    9.12781  2.47%   122 170852s
 3526608 1018671    9.24941  117   26    9.35906    9.12782  2.47%   122 170903s
 3527139 1018954    9.23403   82   32    9.35906    9.12782  2.47%   122 170955s
 3527705 1019198    9.13623   81   50    9.35906    9.12783  2.47%   122 171028s
 3528541 1019416    9.15610   83  314    9.35906    9.12783  2.47%   122 171083s
 3529105 1019587 infeasible  101         9.35906    9.12784  2.47%   122 171122s
 3529463 1019815    9.22616   57   59    9.35906    9.12784  2.47%   122 171177s
 3529959 1019984     cutoff   81         9.35906    9.12784  2.47%   122 171225s
 3530491 1020306    9.31896  124  103    9.35906    9.12785  2.47%   122 171281s
 3531034 1020671    9.30996  104    7    9.35906    9.12786  2.47%   122 171347s
 3531817 1020934     cutoff 

 3581697 1040617    9.24385   76   50    9.35906    9.12837  2.46%   122 176267s
 3582170 1040768    9.21304   66   29    9.35906    9.12838  2.46%   122 176308s
 3582479 1040973    9.17151  145   54    9.35906    9.12838  2.46%   122 176370s
 3583108 1041155    9.27460   70   45    9.35906    9.12839  2.46%   122 176430s


In [ ]:
print("The objective value is",m.objval)

# retrieve the districts and their populations
districts = [ [i for i in G.nodes if x[i,j].x > 0.5] for j in range(k)]
district_counties = [ [ G.nodes[i]["NAME20"] for i in districts[j] ] for j in range(k)]
district_populations = [ sum(G.nodes[i]["TOTPOP"] for i in districts[j]) for j in range(k) ]

# print district info
for j in range(k):
    print("District",j,"has population",district_populations[j],"and contains counties",district_counties[j])
    print("")

In [ ]:
# Let's draw it on a map
import geopandas as gpd

In [ ]:
# Read shapefile from "<state>_<level>.shp"
filename = state + '_' + level + '.shp'

# Read geopandas dataframe from file
df = gpd.read_file( filepath + filename )

In [ ]:
# Which district is each county assigned to?
assignment = [ -1 for i in G.nodes ]

labeling = { i : j for i in G.nodes for j in range(k) if x[i,j].x > 0.5 }

# Now add the assignments to a column of the dataframe and map it
node_with_this_geoid = { G.nodes[i]['GEOID20'] : i for i in G.nodes }

# pick a position u in the dataframe
for u in range(G.number_of_nodes()):
    
    geoid = df['GEOID20'][u]
    
    # what node in G has this geoid?
    i = node_with_this_geoid[geoid]
    
    # position u in the dataframe should be given
    # the same district # that county i has in 'labeling'
    assignment[u] = labeling[i]
    
# now add the assignments to a column of our dataframe and then map it
df['assignment'] = assignment

my_fig = df.plot(column='assignment').get_figure()